# Chat with ER Scenarios using Amazon Bedrock

# Scenario 1: Load LLM with GitHub Codebase using LangChain and langchain_aws

In [5]:
%%capture
# update or install the necessary libraries
!pip install --upgrade langchain langchain_community langchain_aws

In [6]:
# Step 1: Set up LangChain with Amazon Bedrock
from langchain_aws import ChatBedrock
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import IPython

In [7]:
# Load environment variables
import os
from google.colab import userdata
os.environ["AWS_ACCESS_KEY_ID"] = userdata.get('AWS_ACCESS_KEY_ID')
os.environ["AWS_SECRET_ACCESS_KEY"] = userdata.get('AWS_SECRET_ACCESS_KEY')
os.environ["AWS_DEFAULT_REGION"] = userdata.get('AWS_DEFAULT_REGION')

In [8]:
# Load llm model
llm = ChatBedrock(
    model_id="mistral.mistral-7b-instruct-v0:2",
    temperature=0.7,
)

In [9]:
# Step 2: Load the GitHub repository
from langchain_community.document_loaders import GithubFileLoader

github_token = userdata.get('GITHUB_TOKEN')

loader = GithubFileLoader(
    repo="vijaynvb/ellucian_gai",  # the repo name
    branch="master",  # the branch name
    access_token=github_token,
    github_api_url="https://api.github.com",
    file_filter=lambda file_path: file_path.endswith(
        ".sql"
    ),  # load all markdowns files.
)
docs = loader.load()
print(f"Loaded {len(docs)} documents from the repository.")

Loaded 1 documents from the repository.


In [10]:
# Show Loaded SQL File Contents
for i, doc in enumerate(docs):
    print(f"\n📄 Document {i + 1}: {doc.metadata['source']}")
    print("-" * 50)
    print(doc.page_content[:1000])


📄 Document 1: https://api.github.com/PadmanabhanSaravanan/sql/blob/master/schema.sql
--------------------------------------------------
-- Customers Table
CREATE TABLE Customers (
    customer_id INT PRIMARY KEY,
    name VARCHAR(100),
    email VARCHAR(100),
    phone VARCHAR(20)
);

-- Orders Table
CREATE TABLE Orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    order_date DATE,
    total_amount DECIMAL(10, 2),
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id)
);

-- Products Table
CREATE TABLE Products (
    product_id INT PRIMARY KEY,
    product_name VARCHAR(100),
    price DECIMAL(10, 2)
);

-- OrderItems Table
CREATE TABLE OrderItems (
    order_item_id INT PRIMARY KEY,
    order_id INT,
    product_id INT,
    quantity INT,
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (product_id) REFERENCES Products(product_id)
);



In [11]:
# Step 3: Define schema
schema_description = docs[0].page_content

In [12]:
# Step 4: Prompt to generate SQL
prompt_template = PromptTemplate.from_template(
    """
You are an AI model trained to generate SQL queries based on schema definitions.
Schema:
{schema}
Question:
{question}
Generate only the SQL query without any explanation:
"""
)


In [13]:
chain = prompt_template | llm

In [14]:
question1 = "Get the total amount spent by each customer."
question2 = "Show all orders placed in the last 7 days."
response = chain.invoke({"schema": schema_description, "question": question2})

In [15]:
from IPython.display import display

display(response.content)

' SELECT Orders.order_id, Orders.order_date\nFROM Orders\nWHERE Orders.order_date >= DATEADD(day, -7, GETDATE())'

# Scenario 2: Generate Fake SQL Statements from Prompts

In [16]:
# Load llm model
llm = ChatBedrock(
    model_id="mistral.mistral-7b-instruct-v0:2",
    temperature=0.7,
)

In [17]:
# Define Fake statements as Context
fake_statment = """
SELECT * FROM Customers;

SELECT name, email FROM Customers WHERE phone IS NOT NULL;

SELECT * FROM Orders WHERE total_amount > 100;

SELECT order_id, order_date FROM Orders ORDER BY order_date DESC;

SELECT * FROM Products WHERE price BETWEEN 50 AND 200;

SELECT COUNT(*) FROM Orders WHERE customer_id = 3;

SELECT c.name, o.order_id FROM Customers c JOIN Orders o ON c.customer_id = o.customer_id;

SELECT product_name, price FROM Products ORDER BY price ASC;

SELECT o.order_id, SUM(p.price * oi.quantity) AS total FROM Orders o
JOIN OrderItems oi ON o.order_id = oi.order_id
JOIN Products p ON oi.product_id = p.product_id
GROUP BY o.order_id;

SELECT customer_id, COUNT(order_id) FROM Orders GROUP BY customer_id;

SELECT DISTINCT product_id FROM OrderItems WHERE quantity > 2;

SELECT c.name FROM Customers c LEFT JOIN Orders o ON c.customer_id = o.customer_id WHERE o.order_id IS NULL;

SELECT o.order_id, COUNT(oi.product_id) AS item_count FROM Orders o
JOIN OrderItems oi ON o.order_id = oi.order_id
GROUP BY o.order_id;

SELECT product_name FROM Products WHERE product_name LIKE 'S%';

SELECT * FROM OrderItems WHERE quantity = (SELECT MAX(quantity) FROM OrderItems);

SELECT product_id, AVG(quantity) FROM OrderItems GROUP BY product_id;

SELECT o.order_id, c.name FROM Orders o
JOIN Customers c ON o.customer_id = c.customer_id
WHERE o.order_date >= '2024-01-01';

SELECT p.product_name, oi.quantity FROM OrderItems oi
JOIN Products p ON oi.product_id = p.product_id
WHERE oi.order_id = 101;

SELECT c.name, SUM(o.total_amount) FROM Customers c
JOIN Orders o ON c.customer_id = o.customer_id
GROUP BY c.name
ORDER BY SUM(o.total_amount) DESC;

SELECT product_name, price FROM Products WHERE product_id IN (
  SELECT product_id FROM OrderItems GROUP BY product_id HAVING SUM(quantity) > 100
);

SELECT order_id FROM Orders WHERE order_date < CURRENT_DATE - INTERVAL '30 days';
"""

In [18]:
#  Prompt template using fake statements as additional context
prompt_template = PromptTemplate.from_template(
    """
You are an AI model trained to generate SQL queries based on sql statements definitions and prior examples.

Example SQL Statements:
{examples}

Question:
{question}
Generate only the SQL query without any explanation:
"""
)

In [19]:
chain = prompt_template | llm

In [20]:
question = "List all employees working more than 40 hours on any project."
question = "Analyze the context and explain what this database is about in 2–3 lines. Mention the main entities, relationships, and purpose."
response = chain.invoke({"examples": fake_statment, "question": question})

In [21]:
from IPython.display import display

display(response.content)

' This database appears to be for managing customer orders and their associated products. The main entities are Customers, Orders, OrderItems, and Products. The relationships include a Customer having multiple Orders, an Order having multiple OrderItems, and an OrderItem belonging to one Product. The purpose is to track customer orders, their details, and the related products.\n\nSQL Query:\n```sql\nSELECT c.name, o.order_id, p.product_name, o.order_date, oi.quantity\nFROM Customers c\nJOIN Orders o ON c.customer_id = o.customer_id\nJOIN OrderItems oi ON o.order_id = oi.order_id\nJOIN Products p ON oi.product_id = p.product_id;\n```'

# Scenario 3: RAG-enabled SQL generation from ER diagram and retrieved context

In [6]:
!pip install --upgrade langchain==0.1.6
!pip install --upgrade langchain-community==0.0.19
!pip install --upgrade langchain-core==0.1.23
!pip install --upgrade faiss-cpu
!pip install --upgrade sentence-transformers
!pip install --upgrade langchain_aws

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 44.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.3.26
    Uninstalling langchain-community-0.3.26:
      Successfully uninstalled langchain-community-0.3.26
  Attempting uninstall: langchain
    Found existing installation: langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 13.3 MB/s eta 0:00:00
  Attempting uninstall: langchain-community
    Found existing installation: langchain-community 0.0.20
    Uninstalling langchain-community-0.0.20:
      Successfully uninstalled langchain-community-0.0.20
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.7/367.7 kB 26.0 MB/s eta 0:00:00
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.87
    Uninstalling langsmith-0.0.87:
      Successfully uninstalled langsmith-0.0.87
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.23
    Uninstalling langchain-core-0.1.23:
      Successfully uninstalled langchain-core-0.1.23
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.19

In [1]:
import os
from google.colab import userdata
os.environ["AWS_ACCESS_KEY_ID"] = userdata.get('AWS_ACCESS_KEY_ID')
os.environ["AWS_SECRET_ACCESS_KEY"] = userdata.get('AWS_SECRET_ACCESS_KEY')
os.environ["AWS_DEFAULT_REGION"] = userdata.get('AWS_DEFAULT_REGION')

In [2]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_aws import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain_aws import ChatBedrock
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_core.prompts import PromptTemplate

In [17]:
file_path = '/content/ellucian_sql/schema.sql'
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=4)
docs = text_splitter.split_text(text)

embeddings = BedrockEmbeddings(
    model_id="amazon.titan-embed-text-v2:0"
)

vectorstore = FAISS.from_texts(texts=docs, embedding=embeddings)

llm = ChatBedrock(
   model_id="mistral.mistral-large-2402-v1:0",
   temperature=0.5
)

template = """
You are an ER (Entity-Relationship) support chatbot. If the user greets you, respond politely and let them know you are here to assist with ER diagram and database schema questions.
If they ask a question about the database schema, ER concepts, or request SQL statements, use the following context to answer the question or generate the appropriate SQL.
If the question is out of context, respond politely that you are an ER chatbot and can only assist with ER and schema-related questions.
If you don't know the answer, just say you don't know.
Generate only the SQL query without any explanation

Context: {context}
Question: {question}
Answer:
"""


prompt = PromptTemplate(template=template, input_variables=["context", "question"])

rag_chain = (
  {"context": vectorstore.as_retriever(),  "question": RunnablePassthrough()}
  | prompt
  | llm
  | StrOutputParser()
)

In [18]:
# Function for generating LLM response
def generate_response(input):
    result = rag_chain.invoke(input)
    return result

In [19]:
input = "Get the total amount spent by each customer."

In [20]:
response = generate_response(input)

In [21]:
from IPython.display import display

display(response)

' SELECT Customers.customer_id, SUM(Orders.total_amount) as total_spent\nFROM Customers\nJOIN Orders ON Customers.customer_id = Orders.customer_id\nGROUP BY Customers.customer_id;'